In [23]:
from pathlib import Path
import json
from functools import reduce
import math
import datetime as dt
import pytz 
from itertools import product
from collections import OrderedDict
import time

import requests
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely.ops as so

ROOT = Path('../')
DATA_DIR = ROOT/'data'

CRS_NZGD49 = {'init': 'epsg:27200', 'no_defs': True}
CRS_NZTM = {'init': 'epsg:2193', 'no_defs': True}
CRS_WGS84 = {'init': 'epsg:4326'}

%matplotlib inline

In [ ]:
# Prepare area unit table

path = DATA_DIR/'raw'/'Geographical Table.csv'
f = pd.read_csv(path, dtype={'SAU': str})
f = f.rename(columns={
    'SAU': 'au2001', 
    'SAU.Desc': 'au_name', 
    'TA': 'territory',
    'Region': 'region',
})
del f['Water']
f.head()

path = DATA_DIR/'raw'/'Market Rent Areas.csv'
g = pd.read_csv(path, dtype={'SAU': str})
g = g.rename(columns={
    'SAU': 'au2001', 
    'MARKET RENT DESCRIPTION': 'rental_area',
    'TA': 'territory',
    'AU NAME': 'au_name',
})

# Clean rental areas
def clean(x):
    y = x.split(' - ')
    y = y[1] if 'District' not in y[1] else y[0]
    return y

g['rental_area'] = g['rental_area'].map(clean)


f = f.merge(g[['au2001', 'rental_area']])

path = DATA_DIR/'au2001.csv'
f.to_csv(str(path), index=False)
f.head()

# Prepare geodata as GeoJSON

In [ ]:
# Read Shapefile

path = DATA_DIR/'raw'/'NZ_AU01_region_simplified'/'NZ_AU01_region.shp'
au = gpd.read_file(str(path))
au.crs = CRS_NZGD49
au = au.to_crs(CRS_WGS84)
au = au.rename(columns={'AU01': 'au2001', 'AU_DESC': 'au_name'})
print(au.shape)
print(au.head())
au.head().plot()


In [ ]:
# Remove water area units

pattern = r'ocean|strait|inlet|harbour'
cond = au['au_name'].str.contains(pattern, case=False)
au = au[~cond].copy()
print(au.shape)
au.head().plot()


In [ ]:
# Merge geodata and metadata, drop null regions, and write to file

path = DATA_DIR/'au2001.csv'
f = pd.read_csv(path, dtype={'au2001': str})

g = au.merge(f[['au2001', 'territory', 'region', 'rental_area']])
g = g[g['region'].notnull()].copy()

path = DATA_DIR/'au2001.geojson'
with path.open('w') as tgt:
    tgt.write(g.to_json())

g.head()

# Create geodata for rental areas 

In [ ]:
# Dissolve area units by area unit group

path = DATA_DIR/'au2001.geojson'
au = gpd.read_file(str(path))

ra = au[['rental_area', 'region', 'territory', 'geometry']].dissolve(by='rental_area').reset_index()

path = DATA_DIR/'rental_areas.geojson'
with path.open('w') as tgt:
    tgt.write(ra.to_json())

ra.head()

# Prepare rent data

In [ ]:
# Reshape and merge all rent data sets

def clean(f, name):
    f = f.copy()
    f = f.rename(columns={
        'SAU': 'au2001',
        'Property_Type': 'property_type',
        'Bedrooms': '#bedrooms'
    })

    # Drop subtotals
    cond = False
    for col in ['au2001', 'property_type', '#bedrooms']:
        cond |= f[col].str.contains('total', case=False)

    f = f[~cond].copy()
    
    # Reshape
    id_vars = ['au2001', 'property_type', '#bedrooms']
    value_vars = [c for c in f.columns if '-' in c]
    f = pd.melt(f, id_vars=id_vars, value_vars=value_vars,
      var_name='quarter', value_name=name)
    
    return f

paths = [
    DATA_DIR/'raw'/'Detailed Bonds Lodged.csv',
    DATA_DIR/'raw'/'Detailed Mean Rents.csv',
    DATA_DIR/'raw'/'Detailed Geomean Rents.csv',
    DATA_DIR/'raw'/'Detailed Synthetic Lower Quartile Rents.csv',
    DATA_DIR/'raw'/'Detailed Synthetic Upper Quartile Rents.csv',
]
names = ['rent_count', 'rent_mean', 'rent_geo_mean', 'rent_synthetic_lower_quartile', 'rent_synthetic_upper_quartile']
frames = []
for path, name in zip(paths, names):
    f = pd.read_csv(path, dtype={'SAU': str})
    frames.append(clean(f, name))
    
f = reduce(lambda x, y: pd.merge(x, y), frames)

# Merge in region data
path = DATA_DIR/'au2001.csv'
g = pd.read_csv(path, dtype={'au2001': str})
f = f.merge(g)

# Write to file
path = DATA_DIR/'rents.csv'
f.to_csv(str(path), index=False)
f[f['rent_count'].notnull()].head()

# Explore rents

In [105]:
path = DATA_DIR/'rents.csv'
rents = pd.read_csv(path, dtype={'au2001': str})
rents.head()


,au2001,property_type,#bedrooms,quarter,rent_count,rent_mean,rent_geo_mean,rent_synthetic_lower_quartile,rent_synthetic_upper_quartile,au_name,territory,region,rental_area
0,500100,House,2,1993-03-01,NaN,NaN,NaN,NaN,NaN,Awanui,Far North District,Northland,Rural Far North
1,500100,House,3,1993-03-01,NaN,NaN,NaN,NaN,NaN,Awanui,Far North District,Northland,Rural Far North
2,500100,House,2,1993-06-01,NaN,NaN,NaN,NaN,NaN,Awanui,Far North District,Northland,Rural Far North
3,500100,House,3,1993-06-01,NaN,NaN,NaN,NaN,NaN,Awanui,Far North District,Northland,Rural Far North
4,500100,House,2,1993-09-01,NaN,NaN,NaN,NaN,NaN,Awanui,Far North District,Northland,Rural Far North


In [106]:
# Slice in time and aggregate 

def hits(group):
    d = {}
    d['hit_frac'] = group['rent_count'].dropna().shape[0]/group['rent_count'].shape[0]
    return pd.Series(d)

def aggregate_rents(f, date, groupby_cols=('rental_area', '#bedrooms')):
    """
    """
    cond = f['quarter'] >= date
    f = f[cond].copy()
    
    def my_agg(group):
        d = {}
        d['territory'] = group['territory'].iat[0]
        d['region'] = group['region'].iat[0]
        d['rent_count'] = group['rent_count'].sum()
        d['rent_mean'] = (group['rent_mean']*group['rent_count']).sum()/d['rent_count']
        d['rent_geo_mean'] = (group['rent_geo_mean']**(group['rent_count']/d['rent_count'])).prod()
        return pd.Series(d)

    g = f.groupby(groupby_cols).apply(my_agg).reset_index()
    return g

agg_rents = aggregate_rents(rents, '2016-12-01')
agg_rents.head()

,rental_area,#bedrooms,region,rent_count,rent_geo_mean,rent_mean,territory
0,Addington,1,Canterbury,133.0,214.863943,218.654135,Christchurch City
1,Addington,2,Canterbury,56.0,339.402991,342.535714,Christchurch City
2,Addington,3,Canterbury,54.0,414.203127,418.796296,Christchurch City
3,Addington,4,Canterbury,5.0,488.000000,494.000000,Christchurch City
4,Addington,5+,Canterbury,NaN,NaN,NaN,Christchurch City


In [111]:
# What fraction of rental data do we have by #bedrooms?

date = '2016-09-01'
f = aggregate_rents(rents, date, groupby_cols=('au2001', '#bedrooms'))
cond = f['region'] == 'Auckland'
print('census area units')
print(f[cond].copy().groupby('#bedrooms').apply(hits).reset_index())

f = aggregate_rents(rents, date, groupby_cols=('rental_area', '#bedrooms'))
cond = f['region'] == 'Auckland'
print('rental area units')
print(f[cond].copy().groupby('#bedrooms').apply(hits).reset_index())


census area units
  #bedrooms  hit_frac
0         1  0.396226
1         2  0.662791
2         3  0.843750
3         4  0.602305
4        5+  0.115016
rental area units
  #bedrooms  hit_frac
0         1  0.767677
1         2  0.959596
2         3  1.000000
3         4  0.919192
4        5+  0.252525


# Choose representative points for rental areas using property titles

In [ ]:
path = DATA_DIR/'rental_areas.geojson'
ra = gpd.read_file(str(path))

path = DATA_DIR/'property_titles.geojson'
t = gpd.read_file(str(path))
t.head()

In [ ]:
%time f = gpd.sjoin(t[['geometry', 'fid']], ra, op='intersects')
f.head()

In [ ]:
def pt(group):
    d = {}
    d['geometry'] = so.unary_union(group['geometry']).representative_point()
    d['territory'] = group['territory'].iat[0]
    d['region'] = group['region'].iat[0]
    return pd.Series(d)

g = gpd.GeoDataFrame(f.groupby('rental_area').apply(pt).reset_index())

path = DATA_DIR/'rental_area_points.geojson'
with path.open('w') as tgt:
    tgt.write(g.to_json())

g.head()

In [ ]:
g[g['region'] == 'Auckland']

# Use Google Maps to compute times and distance matrices




In [175]:
def get_secret(key, secrets_path=ROOT/'secrets.json'):
    secrets_path = Path(secrets_path)
    with secrets_path.open() as src:
        secrets = json.load(src)
    return secrets[key]

GOOGLE_MATRIX_URL = "https://maps.googleapis.com/maps/api/distancematrix/json"
GOOGLE_KEY = get_secret('GOOGLE_API_KEY_ALEX')

def get_matrix(origins, destinations, mode, departure_time=None, url=GOOGLE_MATRIX_URL, key=GOOGLE_KEY, 
  timezone='Pacific/Auckland'):
    """
    Call Google to compute the duration-distance matrix from the list of origins to the list of destinations
    by the given mode at the given departure time.
    
    INPUT:
        origins
            List of WGS84 longitude-latitude pairs that will be round to 5 decimal places 
        destinations
            List of WGS84 longitude-latitude pairs that will be round to 5 decimal places 
        mode
            String; one of 'driving', 'walking', 'bicycling', or 'transit'
        departure_time
            Optional; ISO 8601 datetime string or 'now'; e.g. '2017-06-01T07:30:00'; can't be from the past
        timezone
            String; timezone for query, e.g. 'Pacific/Auckland'; see https://en.wikipedia.org/wiki/List_of_tz_database_time_zones 
            
    OUTPUT:
        A decoded JSON string described at https://developers.google.com/maps/documentation/distance-matrix/intro#DirectionsResponseElements .
    NOTES:
        - If ``departure_time`` is not null and ``mode='driving'``, then each request must contain at most 100 elements, 
          where the number of elements equals the product of the number of origins and number of destinations.
    """
    valid_modes = ['driving', 'walking', 'bicycling', 'transit']
    if mode not in valid_modes:
        raise ValueError('mode must be one of {!s}'.format(valid_modes))
    
    origs = '|'.join(["{:.05f},{:.05f}".format(lat, lon) for lon, lat in origins])
    dests = '|'.join(["{:.05f},{:.05f}".format(lat, lon) for lon, lat in destinations])
    if departure_time not in [None, 'now']:
        tz = pytz.timezone(timezone)
        departure_time = dt.datetime.strptime(departure_time, '%Y-%m-%dT%H:%M:%S')
        departure_time = int(tz.localize(departure_time).timestamp())
        
    params = {
        'origins': origs,
        'destinations': dests,
        'key': key,
        'mode': mode,
        'departure_time': departure_time,
    }
    r = requests.get(url, params=params)

    # Raise an error if bad request
    r.raise_for_status()

    return r.json()         

def matrix_to_df(matrix, orig_names=None, dest_names=None):
    """
    Given a (decoded) JSON time-distance matrix of the form output by :func:``get_matrix``, 
    a list of origin names (defaults to [0, 1, 2, etc.]), 
    and a list of destination names (defaults to [0, 1, 2, etc.]), convert the matrix to a DataFrame with
    the columns:
    
    - ``'origin'``: one of ``orig_names``
    - ``'destination'``: one of ``dest_names``
    - ``'duration'``: time from origin to destination
    - ``'distance'``: distance from origin to destination
    
    The origin and destination names should be listed in the same order as the 'sources' and 'targets' 
    attributes of ``matrix``, respectively.
    """
    # Initialize DataFrame
    columns = ['orig', 'orig_name', 'dest', 'dest_name', 'duration', 'distance']
    f = pd.DataFrame([], columns=columns)
    
    # Append origins and destinations
    origs, dests =  zip(*product(matrix['origin_addresses'], matrix['destination_addresses']))
    f['orig'] = origs
    f['dest'] = dests
    if orig_names is not None and dest_names is not None:
        orig_names, dest_names = zip(*product(orig_names, dest_names))
        f['orig_name'] = orig_names
        f['dest_name'] = dest_names
        
    # Append durations and distances
    if 'duration_in_traffic' in matrix['rows'][0]['elements'][0]:
        dur_key = 'duration_in_traffic'
    else:
        dur_key = 'duration'
    durs = []
    dists = []
    for r in matrix['rows']:
        for e in r['elements']:
            if e['status'] == 'OK':
                durs.append(e[dur_key]['value'])
                dists.append(e['distance']['value'])
            else:
                durs.append(None)
                dists.append(None)
    f['duration'] = durs
    f['distance'] = dists

    return f

def build_matrix(rental_area_points, mode, departure_time=None, chunk_size=100, 
  url=GOOGLE_MATRIX_URL, key=GOOGLE_KEY):
    """
    Compute the duration-distance matrix between all pairs of rental area points given,
    but skip the diagonal entries, that is, the ones with origin equal to destination.
    To do this, call:func:`get_matrix` repeatedly.
    Group the duration-distance calls into ``chunk_size``-to-1 chunks. 
    
    INPUT:
        rental_area_points
            GeoDataFrame
        mode
            See :func:`get_matrix`
        departure_time
            See :func:`get_matrix`
        chunk_size
            Max number of origin-destination rows per matrix query
        url
            See :func:`get_matrix`
        key
            See :func:`get_matrix`
            
    OUTPUT:
        A DataFrame of the form...
        
    NOTES:
        - Sleeps for 1 second after every call to :func:`get_matrix` to stay within API usage limits
    """
    f = rental_area_points.copy()
    frames = []
    status = 'OK'
    for __, row in f.iterrows():
        # Quit if bad status
        if status != 'OK':
            print('Quitting because of bad status:', status)
            break
            
        # Set the single destination
        dests = [row['geometry'].coords[0]]  
        ra = row['rental_area']
        dest_names = [ra]
        
        # Create origin chunks and compute matrix for each chunk to destination 
        ff = f[f['rental_area'] != ra].copy()
        num_chunks = math.ceil(ff.shape[0]/chunk_size)
        for g in np.array_split(ff, num_chunks):
            # Get origins
            origs = [geo.coords[0] for geo in g['geometry']] 
            orig_names = g['rental_area'].values 
            # Get matrix
            try:
                j = get_matrix(origs, dests, mode=mode, departure_time=departure_time, url=url, key=key)
                status = j['status']
                if status != 'OK':
                    break
                df = matrix_to_df(j, orig_names, dest_names)
            except:
                df = pd.DataFrame()
                df['orig'] = np.nan
                df['orig_name'] = orig_names
                df['dest'] = np.nan
                df['dest_name'] = ra
                df['duration'] = np.nan
                df['distance'] = np.nan
            frames.append(df)
            time.sleep(1)
            
    return pd.concat(frames).sort_values(['orig', 'dest'])


In [80]:
# Test some
origs = [
    [174.66339111328125, -36.45000844447082], 
    [174.76158142089844, -36.86533886128865],
    [174.85633850097656, -37.20517535620264],
]
dests = origs[:2]
matrix = get_matrix(origs, dests, mode='transit', departure_time='2017-06-01T08:00:00')
matrix_to_df(matrix, ['bingo', 'bongo', 'boom'], ['bingo', 'bongo'])

{'destination_addresses': ['173 Cowan Bay Rd, Warkworth 0983, New Zealand',
  '-36.86534,174.76158'],
 'origin_addresses': ['173 Cowan Bay Rd, Warkworth 0983, New Zealand',
  '-36.86534,174.76158',
  '-37.20518,174.85634'],
 'rows': [{'elements': [{'distance': {'text': '1 m', 'value': 0},
     'duration': {'text': '1 min', 'value': 0},
     'status': 'OK'},
    {'status': 'ZERO_RESULTS'}]},
  {'elements': [{'status': 'ZERO_RESULTS'},
    {'distance': {'text': '1 m', 'value': 0},
     'duration': {'text': '1 min', 'value': 0},
     'status': 'OK'}]},
  {'elements': [{'status': 'ZERO_RESULTS'}, {'status': 'ZERO_RESULTS'}]}],
 'status': 'OK'}

In [91]:
j = {'origin_addresses': ['19 Margan Ave, New Lynn, Auckland 0600, New Zealand', '72-74 Remuera Rd, Remuera, Auckland 1050, New Zealand', '126 Symonds St, Eden Terrace, Auckland 1010, New Zealand', '37 Rosedale Rd, Pinehill, Auckland 0632, New Zealand', '7 Jordan Ave, Onehunga, Auckland 1061, New Zealand', '-36.6194,174.70263', '102-116 Church St, Otahuhu, Auckland 1062, New Zealand', '15B Sandra Ave, Otara, Auckland 2023, New Zealand', '32A Reeves Rd, Pakuranga, Auckland 2010, New Zealand', '39 Te Koa Rd, Panmure, Auckland 1072, New Zealand', '67 Marne Rd, Papakura, 2110, New Zealand', '7 Dreadon Ave, Papatoetoe, Auckland 2025, New Zealand', '51-53 Birdwood Ave, Papatoetoe, Auckland 2025, New Zealand', '2 Sumner St, Papatoetoe, Auckland 2025, New Zealand', '26 Alberon St, Parnell, Auckland 1052, New Zealand', '335 Mount Wellington Hwy, Mount Wellington, Auckland 1060, New Zealand', "34 O'Neill St, Ponsonby, Auckland 1011, New Zealand", '3 Wakatipu St, Point Chevalier, Auckland 1022, New Zealand', '187-191 Manukau Rd, Pukekohe 2120, New Zealand', '7-9 Glenarden Way, Ranui, Auckland 0612, New Zealand', '117 Victoria Ave, Remuera, Auckland 1050, New Zealand', '17-19 Winhall Rise, Remuera, Auckland 1050, New Zealand', '-36.75845,174.5639', '70 Saddleback Rise, Murrays Bay, Auckland 0630, New Zealand', '43 Haydn Ave, Royal Oak, Auckland 1023, New Zealand', '-37.19671,174.86174', '13A Kesteven Ave, Glendowie, Auckland 1071, New Zealand', '4 Ronald Algie Pl, St Johns, Auckland 1072, New Zealand', '9 Shorwell St, Sandringham, Auckland 1025, New Zealand', '146a Forrest Hill Rd, Forrest Hill, Auckland 0620, New Zealand', '63 The Track, Takanini 2112, New Zealand', '3 Jutland Rd, Hauraki, Auckland 0622, New Zealand', '26-28 Flanshaw Rd, Te Atatu South, Auckland 0610, New Zealand', '8a Shamrock Ln, Te Atatu Peninsula, Auckland 0610, New Zealand', '23 Fearon Ave, Mount Roskill, Auckland 1041, New Zealand', '10-12 Westridge Rd, Titirangi, Auckland 0604, New Zealand', '1 Dunraven Pl, Torbay, Auckland 0630, New Zealand', '4A Whakarite Rd, Ostend, Auckland 1081, New Zealand', '30a Ludlow Terrace, Totara Vale, Auckland 0627, New Zealand', '14-16 Kitchener Rd, Waiuku 2123, New Zealand', '1664-1666 Great North Rd, Avondale, Auckland 1026, New Zealand', '1 Castlehill Ct, Wattle Downs, Auckland 2103, New Zealand', '-36.43191,174.62146', '106 Hobsonville Rd, Hobsonville, Auckland 0618, New Zealand', '-36.88556,174.5421', '44 Western Springs Rd, Western Springs, Auckland 1022, New Zealand', '27 Warwick Ave, Westmere, Auckland 1022, New Zealand', '7 Joshua Pl, Weymouth, Auckland 2103, New Zealand'], 'status': 'OK', 'destination_addresses': ['157 Oteha Valley Rd, Fairview Heights, Auckland 0632, New Zealand'], 'rows': [{'elements': [{'duration': {'text': '1 hour 44 mins', 'value': 6262}, 'distance': {'text': '32.7 km', 'value': 32654}, 'status': 'OK'}]}, {'elements': [{'duration': {'text': '59 mins', 'value': 3558}, 'distance': {'text': '23.3 km', 'value': 23260}, 'status': 'OK'}]}, {'elements': [{'duration': {'text': '1 hour 2 mins', 'value': 3705}, 'distance': {'text': '20.7 km', 'value': 20687}, 'status': 'OK'}]}, {'elements': [{'duration': {'text': '27 mins', 'value': 1612}, 'distance': {'text': '3.2 km', 'value': 3240}, 'status': 'OK'}]}, {'elements': [{'duration': {'text': '1 hour 29 mins', 'value': 5320}, 'distance': {'text': '29.6 km', 'value': 29571}, 'status': 'OK'}]}, {'elements': [{'status': 'ZERO_RESULTS'}]}, {'elements': [{'duration': {'text': '1 hour 33 mins', 'value': 5584}, 'distance': {'text': '38.6 km', 'value': 38580}, 'status': 'OK'}]}, {'elements': [{'duration': {'text': '1 hour 59 mins', 'value': 7140}, 'distance': {'text': '49.3 km', 'value': 49252}, 'status': 'OK'}]}, {'elements': [{'duration': {'text': '1 hour 32 mins', 'value': 5510}, 'distance': {'text': '34.6 km', 'value': 34587}, 'status': 'OK'}]}, {'elements': [{'duration': {'text': '1 hour 31 mins', 'value': 5446}, 'distance': {'text': '31.7 km', 'value': 31661}, 'status': 'OK'}]}, {'elements': [{'duration': {'text': '2 hours 1 min', 'value': 7233}, 'distance': {'text': '51.4 km', 'value': 51432}, 'status': 'OK'}]}, {'elements': [{'duration': {'text': '1 hour 34 mins', 'value': 5627}, 'distance': {'text': '38.3 km', 'value': 38251}, 'status': 'OK'}]}, {'elements': [{'duration': {'text': '1 hour 32 mins', 'value': 5504}, 'distance': {'text': '39.3 km', 'value': 39284}, 'status': 'OK'}]}, {'elements': [{'duration': {'text': '1 hour 30 mins', 'value': 5423}, 'distance': {'text': '38.0 km', 'value': 37972}, 'status': 'OK'}]}, {'elements': [{'duration': {'text': '1 hour 12 mins', 'value': 4334}, 'distance': {'text': '22.4 km', 'value': 22391}, 'status': 'OK'}]}, {'elements': [{'duration': {'text': '1 hour 35 mins', 'value': 5672}, 'distance': {'text': '33.8 km', 'value': 33785}, 'status': 'OK'}]}, {'elements': [{'duration': {'text': '54 mins', 'value': 3268}, 'distance': {'text': '19.2 km', 'value': 19245}, 'status': 'OK'}]}, {'elements': [{'duration': {'text': '1 hour 17 mins', 'value': 4623}, 'distance': {'text': '24.1 km', 'value': 24146}, 'status': 'OK'}]}, {'elements': [{'duration': {'text': '2 hours 19 mins', 'value': 8320}, 'distance': {'text': '69.5 km', 'value': 69483}, 'status': 'OK'}]}, {'elements': [{'duration': {'text': '1 hour 45 mins', 'value': 6300}, 'distance': {'text': '37.7 km', 'value': 37674}, 'status': 'OK'}]}, {'elements': [{'duration': {'text': '1 hour 26 mins', 'value': 5166}, 'distance': {'text': '26.2 km', 'value': 26210}, 'status': 'OK'}]}, {'elements': [{'duration': {'text': '1 hour 43 mins', 'value': 6183}, 'distance': {'text': '28.8 km', 'value': 28766}, 'status': 'OK'}]}, {'elements': [{'status': 'ZERO_RESULTS'}]}, {'elements': [{'duration': {'text': '48 mins', 'value': 2897}, 'distance': {'text': '8.5 km', 'value': 8495}, 'status': 'OK'}]}, {'elements': [{'duration': {'text': '1 hour 29 mins', 'value': 5320}, 'distance': {'text': '28.3 km', 'value': 28265}, 'status': 'OK'}]}, {'elements': [{'status': 'ZERO_RESULTS'}]}, {'elements': [{'duration': {'text': '1 hour 34 mins', 'value': 5626}, 'distance': {'text': '31.3 km', 'value': 31254}, 'status': 'OK'}]}, {'elements': [{'duration': {'text': '1 hour 27 mins', 'value': 5221}, 'distance': {'text': '29.8 km', 'value': 29818}, 'status': 'OK'}]}, {'elements': [{'duration': {'text': '1 hour 16 mins', 'value': 4554}, 'distance': {'text': '25.0 km', 'value': 24974}, 'status': 'OK'}]}, {'elements': [{'duration': {'text': '42 mins', 'value': 2500}, 'distance': {'text': '12.1 km', 'value': 12122}, 'status': 'OK'}]}, {'elements': [{'duration': {'text': '1 hour 58 mins', 'value': 7092}, 'distance': {'text': '49.5 km', 'value': 49549}, 'status': 'OK'}]}, {'elements': [{'duration': {'text': '53 mins', 'value': 3204}, 'distance': {'text': '13.2 km', 'value': 13211}, 'status': 'OK'}]}, {'elements': [{'duration': {'text': '1 hour 32 mins', 'value': 5531}, 'distance': {'text': '32.3 km', 'value': 32269}, 'status': 'OK'}]}, {'elements': [{'duration': {'text': '1 hour 41 mins', 'value': 6066}, 'distance': {'text': '33.3 km', 'value': 33345}, 'status': 'OK'}]}, {'elements': [{'duration': {'text': '1 hour 32 mins', 'value': 5492}, 'distance': {'text': '26.3 km', 'value': 26294}, 'status': 'OK'}]}, {'elements': [{'duration': {'text': '2 hours 0 mins', 'value': 7184}, 'distance': {'text': '39.9 km', 'value': 39851}, 'status': 'OK'}]}, {'elements': [{'duration': {'text': '23 mins', 'value': 1374}, 'distance': {'text': '4.0 km', 'value': 3982}, 'status': 'OK'}]}, {'elements': [{'duration': {'text': '2 hours 10 mins', 'value': 7791}, 'distance': {'text': '50.7 km', 'value': 50695}, 'status': 'OK'}]}, {'elements': [{'duration': {'text': '48 mins', 'value': 2902}, 'distance': {'text': '11.4 km', 'value': 11436}, 'status': 'OK'}]}, {'elements': [{'duration': {'text': '2 hours 51 mins', 'value': 10254}, 'distance': {'text': '94.1 km', 'value': 94082}, 'status': 'OK'}]}, {'elements': [{'duration': {'text': '1 hour 36 mins', 'value': 5765}, 'distance': {'text': '27.7 km', 'value': 27739}, 'status': 'OK'}]}, {'elements': [{'duration': {'text': '1 hour 54 mins', 'value': 6851}, 'distance': {'text': '47.9 km', 'value': 47861}, 'status': 'OK'}]}, {'elements': [{'status': 'ZERO_RESULTS'}]}, {'elements': [{'duration': {'text': '57 mins', 'value': 3401}, 'distance': {'text': '18.6 km', 'value': 18641}, 'status': 'OK'}]}, {'elements': [{'status': 'ZERO_RESULTS'}]}, {'elements': [{'duration': {'text': '1 hour 16 mins', 'value': 4571}, 'distance': {'text': '23.9 km', 'value': 23852}, 'status': 'OK'}]}, {'elements': [{'duration': {'text': '1 hour 19 mins', 'value': 4722}, 'distance': {'text': '22.0 km', 'value': 21970}, 'status': 'OK'}]}, {'elements': [{'duration': {'text': '2 hours 1 min', 'value': 7281}, 'distance': {'text': '49.5 km', 'value': 49537}, 'status': 'OK'}]}]}
matrix_to_df(j)

,orig,orig_name,dest,dest_name,duration,distance
0,"19 Margan Ave, New Lynn, Auckland 0600, New Ze...",NaN,"157 Oteha Valley Rd, Fairview Heights, Aucklan...",NaN,6262.0,32654.0
1,"72-74 Remuera Rd, Remuera, Auckland 1050, New ...",NaN,"157 Oteha Valley Rd, Fairview Heights, Aucklan...",NaN,3558.0,23260.0
2,"126 Symonds St, Eden Terrace, Auckland 1010, N...",NaN,"157 Oteha Valley Rd, Fairview Heights, Aucklan...",NaN,3705.0,20687.0
3,"37 Rosedale Rd, Pinehill, Auckland 0632, New Z...",NaN,"157 Oteha Valley Rd, Fairview Heights, Aucklan...",NaN,1612.0,3240.0
4,"7 Jordan Ave, Onehunga, Auckland 1061, New Zea...",NaN,"157 Oteha Valley Rd, Fairview Heights, Aucklan...",NaN,5320.0,29571.0
5,"-36.6194,174.70263",NaN,"157 Oteha Valley Rd, Fairview Heights, Aucklan...",NaN,NaN,NaN
6,"102-116 Church St, Otahuhu, Auckland 1062, New...",NaN,"157 Oteha Valley Rd, Fairview Heights, Aucklan...",NaN,5584.0,38580.0
7,"15B Sandra Ave, Otara, Auckland 2023, New Zealand",NaN,"157 Oteha Valley Rd, Fairview Heights, Aucklan...",NaN,7140.0,49252.0
8,"32A Reeves Rd, Pakuranga, Auckland 2010, New Z...",NaN,"157 Oteha Valley Rd, Fairview Heights, Aucklan...",NaN,5510.0,34587.0
9,"39 Te Koa Rd, Panmure, Auckland 1072, New Zealand",NaN,"157 Oteha Valley Rd, Fairview Heights, Aucklan...",NaN,5446.0,31661.0


In [173]:
path = DATA_DIR/'rental_area_points.geojson'
f = gpd.read_file(str(path))
f = f[f['region'] == 'Auckland'].copy()
f

,geometry,id,region,rental_area,territory
2,POINT (174.7136142163085 -36.71740768433499),2,Auckland,Albany,North Shore City
6,POINT (174.689014050193 -36.89612845095292),6,Auckland,Avondale,Auckland City
10,POINT (174.7513445789724 -36.89353312125105),10,Auckland,Balmoral,Auckland City
13,POINT (174.6988893987124 -36.79717233861077),13,Auckland,Beachhaven/Birkdale,North Shore City
18,POINT (174.7059674167557 -36.91481011710743),18,Auckland,Blockhouse Bay/New Windsor,Auckland City
19,POINT (174.9213767017827 -36.91701523623668),19,Auckland,Botony Downs,Manukau City
21,POINT (174.7347763337456 -36.71729061740317),21,Auckland,Browns Bay,North Shore City
22,POINT (174.9105111501434 -36.87507566798835),22,Auckland,Bucklands Beach,Manukau City
30,POINT (174.7686467667584 -36.84997406733503),30,Auckland,Central East,Auckland City
34,POINT (174.7603149671217 -36.85150535125068),34,Auckland,Central West,Auckland City


In [174]:
# Test some more
departure_time = '2017-06-01T07:30:00'
m = build_matrix(f[:2], mode='transit', departure_time=departure_time, chunk_size=50, key=get_secret('GOOGLE_API_KEY_SAEID'))
m

,orig,orig_name,dest,dest_name,duration,distance
0,"157 Oteha Valley Rd, Fairview Heights, Aucklan...",Albany,"34 Nacton Ln, Avondale, Auckland 1026, New Zea...",Avondale,4568,28204
0,"34 Nacton Ln, Avondale, Auckland 1026, New Zea...",Avondale,"157 Oteha Valley Rd, Fairview Heights, Aucklan...",Albany,5903,33138


In [20]:
# Estimate cost of job at 0.5/1000 USD/element beyond 2500 elements
def compute_google_cost(n):
    n = f.shape[0]
    d = OrderedDict()
    d['#rental areas'] = n
    N = 4*n**2
    d['#elements needed for 4 modes'] = N
    d['exceeds 100000-element daily limit?'] = N > 100000 
    d['duration for job in minutes'] = (N/100)/60
    d['cost for job in USD'] = (N - 2500)*(0.5/1000)
    return pd.Series(d)

compute_google_cost(f.shape[0])

#rental areas                               98
#elements needed for 4 modes             38416
exceeds 100000-element daily limit?      False
duration for job in minutes            6.40267
cost for job in USD                     17.958
dtype: object

In [96]:
# Build matrices

departure_time='2017-06-01T07:30:00'
key = get_secret('GOOGLE_API_KEY_PHIL')
for mode in ['driving', 'walking', 'bicycling', 'transit']:
    %time m = build_matrix(f, mode=mode, departure_time=departure_time, key=key)
    print(m.head())
    path = DATA_DIR/'auckland'/'{!s}_commutes.csv'.format(mode)
    m.to_csv(str(path), index=False)

CPU times: user 2.02 s, sys: 52 ms, total: 2.07 s
Wall time: 2min 38s
                   orig                        orig_name  \
91  -36.43191,174.62146  Wellsford/Warkworth/Helensville   
91  -36.43191,174.62146  Wellsford/Warkworth/Helensville   
92  -36.43191,174.62146  Wellsford/Warkworth/Helensville   
91  -36.43191,174.62146  Wellsford/Warkworth/Helensville   
91  -36.43191,174.62146  Wellsford/Warkworth/Helensville   

                                                 dest  \
91                                 -36.6194,174.70263   
91                                 -36.75845,174.5639   
92                                 -36.88556,174.5421   
91                                -37.19671,174.86174   
91  1 Castlehill Ct, Wattle Downs, Auckland 2103, ...   

                     dest_name  duration  distance  
91          Orewa/Whangaparaoa       NaN       NaN  
91      Rewiti/Kumeu/Riverhead       NaN       NaN  
92       Western Beaches/Rural       NaN       NaN  
91            

In [102]:
mode = 'transit'
path = DATA_DIR/'auckland'/'{!s}_commutes.csv'.format(mode)
f = pd.read_csv(path)
n = f.shape[0]
k = f[f['distance'].notnull()].shape[0]
print(k, n, k/n)

8554 9506 0.8998527245949927


# Explore Auckland Transport journey planner

In [168]:
def get_journey(orig, dest, departure_time=None, max_walk=2000):
    """
    INPUT:
        orig
            List; WGS84 longitude-latitude pair
        dest
            List; WGS84 longitude-latitude pair
        departure_time
            String; ISO 8601 datetime; e.g. '2017-06-01T07:30:00'

    """
    url = 'https://api.at.govt.nz/v2/public-restricted/journeyplanner/silverRailIVU/plan'
    fromLoc ='{!s},{!s}'.format(orig[1], orig[0])
    toLoc ='{!s},{!s}'.format(dest[1], dest[0])
    date = departure_time + '+12:00'  # Add UTC offset
    params = {
        'from': 'from',
        'to': 'to',
        'fromLoc': fromLoc,
        'toLoc': toLoc,
        'timeMode': 'A',
        'date': date, 
        'modes': 'BUS,TRAIN,FERRY',
        'operators': '',
        'optimize': 'QUICK',
        'maxWalk': max_walk,
        'maxChanges': '-1',
        'routes': '',
        'subscription-key': '323741614c1c4b9083299adefe100aa6',
    }
    r = requests.get(url, params=params)
    
    # Raise an error if bad request
    r.raise_for_status()

    return r.json()         


In [169]:
# Test some
orig = [174.7433853149414, -36.85517522550505]
dest = [174.79625701904297, -36.82550066651677]
j = get_journey(orig, dest)


In [170]:
for leg in j['response']['itineraries'][0]['legs']:
    print(leg['mode'], leg['distance'])

WALK 267 metres
BUS undefined metres
WALK 330 metres
FERRY undefined metres
WALK 1072 metres
